In [1]:
import pandas as pd 

import sys
import os
models_path = os.path.abspath('../data')
sys.path.append(models_path)
import age_from_reviews

In [2]:
reviews_path = "/Users/marijazelic/Downloads/reviews_df.csv"
users_path = "/Users/marijazelic/Downloads/users.csv"

users_reviews = age_from_reviews.AgeFromReviews(users_path, reviews_path)
#us_users_reviews = users_reviews.filter_beer_type()


In [3]:
year_list = [2004, 2008, 2012, 2016]
results = users_reviews.aggregate_preferences_year(year_list)

In [4]:
results

,IPA_2004,Lager_2004,Other Ale_2004,Pale Ale_2004,Pilsner_2004,Porter_2004,Red/Amber Ale_2004,Stout_2004,IPA_2008,Lager_2008,...,Red/Amber Ale_2012,Stout_2012,IPA_2016,Lager_2016,Other Ale_2016,Pale Ale_2016,Pilsner_2016,Porter_2016,Red/Amber Ale_2016,Stout_2016
state,,,,,,,,,,,,,,,,,,,,,
Arizona,4.078716,2.693111,3.854716,3.692993,3.148857,3.934524,3.652069,4.098452,3.907270,3.155522,...,3.731053,4.021909,4.095421,3.397922,4.058177,3.923902,3.746765,3.905172,3.971538,4.265000
California,4.012170,3.013805,3.911051,3.767962,3.533864,3.873772,3.704977,3.979217,3.969729,3.025408,...,3.759672,3.984838,4.100892,3.554089,4.069353,3.973131,3.723706,4.026823,3.908415,4.223051
Florida,3.970366,2.828348,3.924844,3.604157,3.556667,3.754400,3.599000,3.859365,3.995041,2.963718,...,3.804307,4.032356,4.027201,3.394058,3.897478,3.934747,3.705532,4.057823,3.751837,4.167336
Indiana,3.809275,2.982222,3.829592,3.641159,3.517143,4.066875,3.455217,3.986792,3.998911,3.326133,...,3.749524,4.044981,4.053942,3.389770,3.940187,3.951591,3.864333,3.991920,3.803667,4.177562
Nevada,3.930000,3.035484,3.976000,3.748519,3.421667,3.951429,3.425556,4.079048,4.024894,2.989048,...,3.625769,3.971818,3.643165,2.802883,3.628853,3.468274,3.315091,3.692121,3.409900,3.888391
New Hampshire,3.797576,2.984576,3.740909,3.518788,3.295000,3.813333,3.293214,3.879038,4.005721,3.028295,...,3.662987,3.996100,4.033725,3.572627,3.895932,3.869608,3.703962,3.967651,3.740000,4.121224
New York,3.998836,3.037725,3.846911,3.774797,3.557157,3.912759,3.751623,3.986264,3.959608,3.079911,...,3.710708,4.006983,4.139786,3.229683,3.985519,3.919220,3.613880,3.999759,3.786732,4.160219
North Carolina,3.914806,3.202744,3.765630,3.691271,3.539437,3.885776,3.728932,3.925263,4.021210,3.172450,...,3.795051,4.032108,4.081694,3.624104,4.055501,3.957443,3.912532,4.049819,3.985800,4.181333
Ohio,3.980127,2.997788,3.749115,3.669304,3.390562,3.946686,3.701101,3.983634,3.940216,3.101334,...,3.792402,4.044002,3.959815,3.457235,3.944341,3.888231,3.694636,3.959321,3.909661,4.122027
